In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

In [2]:
# tensorflow gpu 사용 가능한지 체크
tf.test.is_gpu_available()

True

In [3]:
# 연산과 텐서가 어떤 디바이스에 배치되었는지 알아보기 위해
# 세션을 만들 때 log_device_placement 옵션을 True로 설정 가능
with tf.Session(config = tf.ConfigProto(log_device_placement=True)) as session:
    pprint.pprint(session.list_devices())

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:09:00.0, compute capability: 7.5

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 14041509942375165425),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 9106889769, 14255716566187076488)]


#### 디바이스 배치 로깅

In [4]:
# 그래프 생성
a = tf.constant([1., 2., 3., 4., 5., 6.], shape=[2, 3], name='a')
b = tf.constant([1., 2., 3., 4., 5., 6.], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# log_device_placement을 True로 설정하여 세션을 만듦
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))
sess.close()

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:09:00.0, compute capability: 7.5

[[22. 28.]
 [49. 64.]]


#### 수동으로 디바이스에 배치

In [5]:
# 그래프를 생성
with tf.device('/cpu:0'):
    a = tf.constant([1., 2., 3., 4., 5., 6.], shape=[2, 3], name='a')
    b = tf.constant([1., 2., 3., 4., 5., 6.], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# log_device_placement를 True로 설정하여 세션을 만듦
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# op를 실행
print(sess.run(c))
sess.close()

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:09:00.0, compute capability: 7.5

[[22. 28.]
 [49. 64.]]


In [6]:
import sys
!git clone https://github.com/google-research/bert

fatal: destination path 'bert' already exists and is not an empty directory.


In [7]:
sys.path += ['./bert']

In [8]:
# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub
import tensorflow_hub as hub

## Prepare for training

#### Experiments: 11개의 NLP Tasks
1. MNLI(Multi-Genre Natural Language Inference): 두 개의 문장을 주고, 두번째 문장이 첫번째 문장과 같은 의미(entailment)인지, 모순(contradiction)이 있는지, 무관한(neutral) 의미인지 판단
2. QQP(Quora Question Pair): 두 질문이 같은 의미의 질문인지 판단
3. QNLI(Question Natural Language Inference): 질문과 문장을 주고, 그 문장에 답이 있으면 pos, 없으면 neg를 판단
4. SST-2(Stanford Sentiment Treebank): movie review data에서 sentiment 분석(binary)
5. CoLA(Corpus of Linguistic Acceptability): 문장의 문법이 맞는지 판단
6. STS-B(Semantic Textual Similarity Benchmark): 두 문장의 sentiment가 같은지 판단
7. MRPC(Microsoft Research Paraphrase corpus): 두 문장의 sentiment가 같은지 판단
8. RTE(Recognizing Textual Entailment): MNLI와 비슷
9. SQuAD 1.1(Stanford Question Answering Database): 질문을 보고 지문에서 answer text span을 찾아낸다. 즉, 정답의 시작점과 끝점을 찾아냄
10. CoNLL Named Entity Recognition: 단어에 Person, Organization, Location, Miscellaneous, Other-Not named entity를 annotate
11. SWAG(Situation with Adversarial Generations): video captioning DB에서 추출한 문장 다음에 올 문장으로 알맞은 것은? (4지선다형)

In [9]:
TASK = 'MRPC' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

In [10]:
# Download glue data.
!git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo

fatal: destination path 'download_glue_repo' already exists and is not an empty directory.


In [11]:
!python download_glue_repo/download_glue_data.py --data_dir=glue_data --tasks=$TASK

Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!


In [12]:
TASK_DATA_DIR = 'glue_data\\' + TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!dir $TASK_DATA_DIR

***** Task data directory: glue_data\MRPC *****
 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 1687-9ECC

 C:\workspace\dacon문자스미싱\glue_data\MRPC 디렉터리

2019-12-13  오후 04:37    <DIR>          .
2019-12-13  오후 04:37    <DIR>          ..
2019-12-27  오전 10:53           106,025 dev.tsv
2019-12-27  오전 10:53             6,222 dev_ids.tsv
2019-12-27  오전 10:53           441,275 msr_paraphrase_test.txt
2019-12-27  오전 10:53         1,047,044 msr_paraphrase_train.txt
2019-12-27  오전 10:53           447,061 test.tsv
2019-12-27  오전 10:53           945,140 train.tsv
               6개 파일           2,992,767 바이트
               2개 디렉터리  277,213,212,672 바이트 남음


In [13]:
BUCKET = './bucket/' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'

In [14]:
OUTPUT_DIR = './{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)

In [15]:
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: ././bucket//bert-tfhub/models/MRPC *****


In [16]:
print('BERT_MODEL_HUB\t', BERT_MODEL_HUB)
with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature='tokenization_info',
                                    as_dict=True)
    with tf.Session() as sess:
        vocab_file, do_lower_case = sess.run(
            [tokenization_info['vocab_file'],
             tokenization_info['do_lower_case']])
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

BERT_MODEL_HUB	 https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
vocab_file

b'C:\\Users\\jinma\\AppData\\Local\\Temp\\tfhub_modules\\5a395eafef2a37bd9fc55d7f6ae676d2a134a838\\assets\\vocab.txt'

In [18]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [19]:
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 64
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [21]:
TASK_DATA_DIR

'glue_data\\MRPC'

In [20]:
train_examples

 ...]

## Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

In [17]:
# Force TF Hub writes to the GS bucket we provide
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

In [18]:
OUTPUT_DIR

'././bucket//bert-tfhub/models/MRPC'

In [77]:
def create_model(is_training, input_ids, input_mask, 
                 segment_ids, labels,
                 num_labels, bert_hub_module_handle):
    """Creates a classification model."""
    tags = set()
    if is_training:
        tags.add("train")
    bert_module = hub.Module(bert_hub_module_handle, 
                             tags=tags, trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use
    # bert_outputs["sequence_output"] instead.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
          # I.e., 0.1 dropout
          output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)

In [94]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle):
    def model_fn(features, labels, mode, params):  
        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (
                name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
            bert_hub_module_handle)
        
        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimization.create_optimizer(
                  total_loss, learning_rate, num_train_steps, 
                  num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op)
        elif mode == tf.estimator.ModeKeys.EVAL:
            eval_metrics = {
                'eval_accuracy' : tf.metrics.accuracy(label_ids, 
                                tf.argmax(logits, axis=-1, 
                                          output_type=tf.int32)),
                'eval_loss' : tf.metrics.mean(per_example_loss)
            }
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics)
        elif mode == tf.estimator.ModeKeys.PREDICT:
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode, 
                predictions={"probabilities": probabilities})
        else:
            raise ValueError(
                "Only TRAIN, EVAL and PREDICT modes are supported: %s" % 
                (mode))
        return output_spec

    return model_fn

In [95]:
model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False,
    bert_hub_module_handle=BERT_MODEL_HUB
)

In [96]:
def get_run_config(output_dir):
    return tf.estimator.RunConfig(
        model_dir=output_dir,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [97]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=get_run_config(OUTPUT_DIR),
    params={
        'batch_size' : TRAIN_BATCH_SIZE,
#         'eval_batch_size' : EVAL_BATCH_SIZE,
#         'predict_batch_size' : PREDICT_BATCH_SIZE
    }
)

INFO:tensorflow:Using config: {'_model_dir': '././bucket//bert-tfhub/models/MRPC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000204B3A67EB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '././bucket//bert-tfhub/models/MRPC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000204B3A67EB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [98]:
# Train the model
def model_train(estimator):
    print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
    # We'll set sequences to be at most 128 tokens long.
    train_features = run_classifier.convert_examples_to_features(
        train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    print('***** Started training at {} *****'.format(datetime.datetime.now()))
    print('  Num examples = {}'.format(len(train_examples)))
    print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
    tf.logging.info('  Num steps = %d', num_train_steps)
    train_input_fn = run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=True)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [99]:
model_train(estimator)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 3668


INFO:tensorflow:Writing example 0 of 3668


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: train-1


...

***** Started training at 2019-12-13 19:30:06.926777 *****
  Num examples = 3668
  Batch size = 16
INFO:tensorflow:  Num steps = 687


INFO:tensorflow:  Num steps = 687


INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


***** Finished training at 2019-12-13 19:30:06.937780 *****


In [100]:
def model_eval(estimator):
    # Eval the model
    eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
    eval_features = run_classifier.convert_examples_to_features(
        eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
    print('  Num examples = {}'.format(len(eval_examples)))
    print('  Batch size = {}'.format(EVAL_BATCH_SIZE))
    eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
    eval_input_fn = run_classifier.input_fn_builder(
        features=eval_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=True)
    result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
    print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
    output_eval_file = os.path.join(OUTPUT_DIR, 'eval_results.txt')
    with tf.gfile.GFile(output_eval_file, 'w') as writer:
        print('***** Eval results *****')
        for key in sorted(result.keys()):
            print('  {} = {}'.format(key, str(result[key])))

In [101]:
model_eval(estimator)

INFO:tensorflow:Writing example 0 of 408


INFO:tensorflow:Writing example 0 of 408


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-1


...

INFO:tensorflow:Saving dict for global step 687: eval_accuracy = 0.8425, eval_loss = 0.5332047, global_step = 687, loss = 0.5332047


INFO:tensorflow:Saving dict for global step 687: eval_accuracy = 0.8425, eval_loss = 0.5332047, global_step = 687, loss = 0.5332047


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 687: ././bucket//bert-tfhub/models/MRPC\model.ckpt-687


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 687: ././bucket//bert-tfhub/models/MRPC\model.ckpt-687


***** Finished evaluation at 2019-12-13 19:30:11.694768 *****
***** Eval results *****
  eval_accuracy = 0.8425
  eval_loss = 0.5332047
  global_step = 687
  loss = 0.5332047


In [102]:
processor.get_dev_examples(
            TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]

In [103]:
def model_predict(estimator):
    # Make predictions on a subset of eval examples
    prediction_examples = processor.get_dev_examples(
            TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
    input_features = run_classifier.convert_examples_to_features(
            prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features,
                                                       seq_length=MAX_SEQ_LENGTH,
                                                       is_training=False, 
                                                       drop_remainder=True)
    predictions = estimator.predict(predict_input_fn)
    return prediction_examples, predictions
#     for example, prediction in zip(prediction_examples, predictions):
#         print('text_a: {}\ntext_b: {}\nlabel:{}\nprediction:{}\n'.format(
#                 example.text_a, example.text_b, 
#                 str(example.label), prediction['probabilities']))

In [104]:
prediction_examples, predictions = model_predict(estimator)

INFO:tensorflow:Writing example 0 of 8


INFO:tensorflow:Writing example 0 of 8


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-1


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [105]:
for example, prediction in zip(prediction_examples, predictions):
    print('text_a: {}\ntext_b: {}\nlabel:{}\nprediction:{}\n'.format(
            example.text_a, example.text_b, 
            str(example.label), prediction['probabilities']))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:*** Features ***


INFO:tensorflow:*** Features ***


INFO:tensorflow:  name = input_ids, shape = (16, 64)


...

INFO:tensorflow:Restoring parameters from ././bucket//bert-tfhub/models/MRPC\model.ckpt-687


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [106]:
prediction_examples[0].text_a

"He said the foodservice pie business doesn 't fit the company 's long-term growth strategy ."

In [91]:
prediction_examples[0].text_b

'" The foodservice pie business does not fit our long-term growth strategy .'

In [92]:
str(prediction_examples[0].label)

'1'

In [93]:
[prediction for i in predictions]

[]